In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [31]:
df = pd.read_csv('/Users/ziadharmanani/Desktop/BoC_BankRate/data/BoC_MoneyMarket.csv')

In [32]:
df.columns.tolist()

['REF_DATE',
 'GEO',
 'DGUID',
 'Financial market statistics',
 'UOM',
 'UOM_ID',
 'SCALAR_FACTOR',
 'SCALAR_ID',
 'VECTOR',
 'COORDINATE',
 'VALUE',
 'STATUS',
 'SYMBOL',
 'TERMINATED',
 'DECIMALS']

In [33]:
# Address datatypes before analytics
df['REF_DATE'] = pd.to_datetime(df['REF_DATE'], errors='coerce')
df['VALUE'] = pd.to_numeric(df['VALUE'], errors='coerce')

# Drop internal identifiers and pure formatting metadata
df = df.drop(columns=['DGUID', 'VECTOR', 'COORDINATE', 'DECIMALS'])

# Minimal renaming for readability
df = df.rename(columns={
    'REF_DATE': 'DATE',
    'Financial market statistics': 'INDICATOR'
})

# Enforce core data types
df['DATE'] = pd.to_datetime(df['DATE'], errors='coerce')
df['VALUE'] = pd.to_numeric(df['VALUE'], errors='coerce')

# Keep only valid numeric observations
df = df.loc[df['VALUE'].notna()].copy()

In [34]:
df.info()
df.head()
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 332326 entries, 12 to 668674
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   DATE           332326 non-null  datetime64[ns]
 1   GEO            332326 non-null  object        
 2   INDICATOR      332326 non-null  object        
 3   UOM            332326 non-null  object        
 4   UOM_ID         332326 non-null  int64         
 5   SCALAR_FACTOR  332326 non-null  object        
 6   SCALAR_ID      332326 non-null  int64         
 7   VALUE          332326 non-null  float64       
 8   STATUS         0 non-null       object        
 9   SYMBOL         0 non-null       float64       
 10  TERMINATED     101522 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(6)
memory usage: 30.4+ MB


DATE                  0
GEO                   0
INDICATOR             0
UOM                   0
UOM_ID                0
SCALAR_FACTOR         0
SCALAR_ID             0
VALUE                 0
STATUS           332326
SYMBOL           332326
TERMINATED       230804
dtype: int64

In [42]:
# Indicator summary (Number ofobservations, start date and end date)
indicator_summary = (
    df
    .groupby('INDICATOR')
    .agg(
        n_obs=('VALUE', 'count'),
        start_date=('DATE', 'min'),
        end_date=('DATE', 'max')
    )
    .sort_values('n_obs', ascending=False)
)

indicator_summary

,n_obs,start_date,end_date
INDICATOR,,,
Bank rate,17006,1960-07-27,2025-11-27
"Treasury Bills, 3-month",16305,1960-07-22,2025-11-27
"Treasury Bills, 1-month",16230,1960-07-22,2025-11-27
"Treasury Bills, 2-month",16229,1960-07-22,2025-11-27
"Treasury Bills, 6-month",14994,1960-07-22,2025-11-27
"Prime corporate paper rate, 3-month",14600,1960-07-27,2018-12-31
"Prime corporate paper rate, 1-month",14596,1960-07-27,2018-12-31
"Prime corporate paper rate, 2-month",14586,1960-07-27,2018-12-31
"Government of Canada benchmark bond yields, long term",14014,1970-01-02,2025-11-27


In [36]:
df['GEO'].value_counts()

GEO
Canada    332326
Name: count, dtype: int64

In [37]:
df[['INDICATOR', 'UOM', 'SCALAR_FACTOR']].drop_duplicates()

,INDICATOR,UOM,SCALAR_FACTOR
12,"Treasury Bills, 1-month",Percent,units
13,"Treasury Bills, 2-month",Percent,units
14,"Treasury Bills, 3-month",Percent,units
15,"Treasury Bills, 6-month",Percent,units
155,"Prime corporate paper rate, 1-month",Percent,units
156,"Prime corporate paper rate, 2-month",Percent,units
157,"Prime corporate paper rate, 3-month",Percent,units
160,Bank rate,Percent,units
22400,"Bankers' acceptances rate, 1-month",Percent,units
23022,"Bankers' acceptances rate, 2-month",Percent,units


In [39]:
df_bank_rate = (df[df['INDICATOR'] == "Bank rate"].sort_values('DATE'))
df_bank_rate = df_bank_rate.set_index('DATE')
df_bank_rate.head(5)

,GEO,INDICATOR,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VALUE,STATUS,SYMBOL,TERMINATED
DATE,,,,,,,,,,
1960-07-27,Canada,Bank rate,Percent,239,units,0,3.44,NaN,NaN,NaN
1960-07-28,Canada,Bank rate,Percent,239,units,0,3.17,NaN,NaN,NaN
1960-07-29,Canada,Bank rate,Percent,239,units,0,3.17,NaN,NaN,NaN
1960-08-01,Canada,Bank rate,Percent,239,units,0,3.17,NaN,NaN,NaN
1960-08-02,Canada,Bank rate,Percent,239,units,0,3.17,NaN,NaN,NaN


In [40]:
df_bank_rate.to_csv('/Users/ziadharmanani/Desktop/BoC_BankRate/data/bank_rate_clean.csv')